# M3 N1 Introduction to bokeh with pandas

In this notebook, we'll introduce the `bokeh` plotting package in `python`, wrangle some cloud-hosted data with `pandas`, and introduce some of the methods you'll use in Module 3's homework.

You'll need to install a few new packages for this assignment, so be sure to install them in the class conda environment.

One is `pandas-bokeh`, which can be installed with

```bash
pip install pandas-bokeh
```

All right, for this notebook, and for your assignment, we'll be using the NOAA Global Historical Climatology Network Daily (GHCN-D) database, which is the official record for daily weather observations.  The dataset is produced by the National Centers for Environmental Information https://doi.org/10.7289/V5D21VHZ.  As part of the NOAA Big Data project, this dataset is hosted in the cloud by several popular cloud data services. For this assignment, we'll use the service on Amazon Web Services, who hosts the dataset in an S3 bucket in a few formats. The link to the S3 page for GHCN-D is https://registry.opendata.aws/noaa-ghcn/.

The data is produced in its original format in fixed-width format text files.  We'll use `pandas` to load some metadata - the station inventory file.

In [15]:
import pandas as pd
stn_ids = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt', header=None, infer_nrows=1000)
stn_ids.columns = ['ID','LAT','LON','ELEV','UKN','NAME','GSN','WBAN']
stn_ids

,ID,LAT,LON,ELEV,UKN,NAME,GSN,WBAN
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,41217.0
...,...,...,...,...,...,...,...,...
127989,ZI000067969,-21.0500,29.3670,861.0,NaN,WEST NICHOLSON,NaN,67969.0
127990,ZI000067975,-20.0670,30.8670,1095.0,NaN,MASVINGO,NaN,67975.0
127991,ZI000067977,-21.0170,31.5830,430.0,NaN,BUFFALO RANGE,NaN,67977.0
127992,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,67983.0


The next file we'll open is the station inventory file.  It describes for each site and each variable, the years that each variable is available.

In [16]:
periods = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-inventory.txt', header=None, infer_nrows=1000)
periods.columns = ['ID','LAT','LON','ELEM','TiMIN','TiMAX']
periods

,ID,LAT,LON,ELEM,TiMIN,TiMAX
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949
...,...,...,...,...,...,...
756420,ZI000067983,-20.2000,32.6160,PRCP,1951,2024
756421,ZI000067983,-20.2000,32.6160,TAVG,1962,2024
756422,ZI000067991,-22.2170,30.0000,TMAX,1951,1990
756423,ZI000067991,-22.2170,30.0000,TMIN,1951,1990


Let's use `pandas` to merge these two tables into one based on the ID.

In [17]:
merged_stns = pd.merge(stn_ids,periods,how='left',left_on='ID',right_on='ID')
merged_stns

,ID,LAT_x,LON_x,ELEV,UKN,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,TMAX,1949.0,1949.0
1,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,TMIN,1949.0,1949.0
2,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,PRCP,1949.0,1949.0
3,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,SNOW,1949.0,1949.0
4,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,SNWD,1949.0,1949.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756430,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,67983.0,-20.2000,32.6160,PRCP,1951.0,2024.0
756431,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,67983.0,-20.2000,32.6160,TAVG,1962.0,2024.0
756432,ZI000067991,-22.2170,30.0000,457.0,NaN,BEITBRIDGE,NaN,67991.0,-22.2170,30.0000,TMAX,1951.0,1990.0
756433,ZI000067991,-22.2170,30.0000,457.0,NaN,BEITBRIDGE,NaN,67991.0,-22.2170,30.0000,TMIN,1951.0,1990.0


Let's filter the merged table just for the `TMAX` element, and those stations still reporting in 2022.  Notice how the number of stations drops - most of the stations in the dataset measure rainfall & many stations have stopped reporting.

In [18]:
#subset for T
merged_stns = merged_stns[(merged_stns['ELEM'] == 'TMAX') & (merged_stns['TiMAX'] == 2022)]
merged_stns


,ID,LAT_x,LON_x,ELEV,UKN,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
182,AGM00060415,36.3170,3.5330,748.0,NaN,AIN-BESSAM,NaN,60415.0,36.3170,3.5330,TMAX,2003.0,2022.0
296,AGM00060518,35.3000,-1.3500,70.0,NaN,BENI-SAF,NaN,60518.0,35.3000,-1.3500,TMAX,1976.0,2022.0
780,AO000066390,-14.9330,13.5670,1761.0,NaN,LUBANGO (SA DA BAND,GSN,66390.0,-14.9330,13.5670,TMAX,1957.0,2022.0
1494,ASN00001018,-16.4181,126.1025,546.0,NaN,MOUNT ELIZABETH,NaN,94211.0,-16.4181,126.1025,TMAX,1993.0,2022.0
2971,ASN00006022,-25.0544,115.2100,144.0,NaN,GASCOYNE JUNCTION,NaN,94410.0,-25.0544,115.2100,TMAX,1750.0,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
744973,USW00092811,25.8064,-80.1333,2.4,FL,MIAMI BEACH,NaN,NaN,25.8064,-80.1333,TMAX,1927.0,2022.0
745189,USW00093027,34.5183,-109.3800,1745.6,AZ,ST JOHNS IND AIR PK,NaN,NaN,34.5183,-109.3800,TMAX,1999.0,2022.0
754281,UV000065501,14.0330,-0.0330,277.0,NaN,DORI,GSN,65501.0,14.0330,-0.0330,TMAX,1973.0,2022.0
756211,WA010517310,-19.1500,15.9120,1100.0,NaN,OKAUKUEJO,NaN,68010.0,-19.1500,15.9120,TMAX,1975.0,2022.0


Let's visualize where these currently reporting stations are using a map, depending on when they started taking measurements.  We'll sort them so the oldest ones are last, so they plot on the top of the map.

In [19]:
merged_stns_sorted = merged_stns.sort_values('TiMIN', ascending=False)

In [20]:
merged_stns_sorted['ELEM']

477384    TMAX
506406    TMAX
619321    TMAX
618322    TMAX
575081    TMAX
          ... 
443525    TMAX
499319    TMAX
662984    TMAX
101658    TMAX
2971      TMAX
Name: ELEM, Length: 302, dtype: object

Let's do a scatter plot on a `cartopy` map.

In [23]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs

fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# make the map global rather than have it zoom in to
# the extents of any plotted data
ax.set_global()
ax.coastlines()

plt.scatter(merged_stns_sorted['LON_x'],merged_stns_sorted['LAT_x'],0.25,c=merged_stns_sorted['TiMIN'], 
            transform=ccrs.PlateCarree(), cmap='magma')
plt.colorbar(shrink = 0.5, label='year')
plt.title('Year of first TMAX measurement in GHCN daily data with stations still reporting in 2022');

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'cartopy'

Let's do a query to find stations that are reporting in 2022 and contain the string 'CHAMPAIGN' the station name field.

In [ ]:
merged_stns[merged_stns['NAME'].str.contains('CHAMPAIGN', regex=False)]

Let's load the data! The data is avaiable in AWS in a couple of formats.  It has a mirror of the raw data format in column separated values (csv), and pandas can read it without a problem.  I've added a couple of flags to avoid errors, but this should work to open the data in this format.

Again, to learn more, visit the AWS site for GHCN-D (https://registry.opendata.aws/noaa-ghcn/)

Let's load 

In [ ]:
df = pd.read_csv(
...      "s3://noaa-ghcn-pds/csv/by_station/USC00118740.csv",
...      storage_options={"anon": True},  # passed to `s3fs.S3FileSystem`
         dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},
         parse_dates=['DATE']
... ).set_index('DATE')

Looking at the structure of the data, each "element" takes up one line, there are QC flags for each variables, and other informtion.

In [ ]:
df

We can also load the data in a parquet format, which is a cloud-optimized database format.  It also works well with `dask`.  We won't use `dask` for this exercise because a single site is a small file.  However, if you were interested in looking at larger portions of the dataset, `dask` could help.

In [ ]:
from datetime import datetime

df = pd.read_parquet(
    "s3://noaa-ghcn-pds/parquet/by_station/STATION=USC00118740/",
    storage_options={"anon": True},  # passed to `s3fs.S3FileSystem`
)

#make date the index
df['DATE'] = df['DATE'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))
df = df.set_index('DATE').sort_index()  #we need to sort by time because the files are sorted to be arbitrary
df


Let's create separate dataframes for elements of interest: Maximum and Minimum Temperature and Precipitaion.

In [ ]:
df_tmax = df.loc[df['ELEMENT'] == 'TMAX']
df_tmin = df.loc[df['ELEMENT'] == 'TMIN']
df_prcp = df.loc[df['ELEMENT'] == 'PRCP']

In [ ]:
df_tmax

Note that the values are a not a magnitude you might expect - they are scaled integers.  To get values in degrees C, divide the values by 10.

## Bokeh and pandas plotting

So, let's visualize this data using `bokeh`.  `bokeh` is an alternative to `matplotlib` and it produces interactive high quality plots that can be visualized in the notebook or on a website.  Check out the `bokeh` website to see what it can do: (https://bokeh.org).

First, we'll use the pandas_bokeh library to plot directly from the pandas dataframe.  We'll also have the plots shown in the notebook here, but you should try turning that off and seeing what happens.  If successful, you should see a little badge below the next cell.

In [25]:
import datetime
from os.path import dirname, join

import pandas as pd
from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure

from ipywidgets import interact
from bokeh.io import show, output_notebook


# see what happens when you comment this out and try to make a bokeh plot!
output_notebook()



Loading BokehJS ...

Ok, let's plot the time series of maximum temperature for the station we loaded above.  We'll add a fancy rangetool to show the data.

In [ ]:
(df_tmax['DATA_VALUE']/10.).plot_bokeh(rangetool=True);

Let's create an annual mean minimum temperature time series using `pandas` groupby.

In [26]:
ser=df_tmin[~((df_tmin.index.month==2)&(df_tmin.index.day==29))]

bk_bar1 = (ser['DATA_VALUE']/10.).groupby(ser.index.day_of_year).mean().plot_bokeh().line()


NameError: name 'df_tmin' is not defined

It's easy to create basing plots using `pandas_bokeh`, but you can also using the `bokeh` interface to make plots similar to a matplotlib-type plot.  Here, let's show the maximum temperatures for 2022.

In [ ]:
# bokeh basics
from turtle import color
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Time series of maximum temperatures - 2022',
          x_axis_label = 'Time', 
           y_axis_label = 'Temperature (°C)',
           x_axis_type='datetime')

df_2022 = df_tmax[df_tmax.index >= '2022-01-01']
p.line(df_2022.index, df_2022['DATA_VALUE']/10., legend_label="TMAX", line_width=2)
df_2_2022 = df_tmin[df_tmin.index >= '2022-01-01']
p.line(df_2_2022.index, df_2_2022['DATA_VALUE']/10., legend_label="TMIN", line_width=2, color='orange')

# Show the plot
show(p)

Let's make a histogram of low temperatures

In [ ]:
import numpy as np
arr_hist, edges = np.histogram(df_tmin['DATA_VALUE'][(df_tmin.index >= '1991-01-01') & (df_tmin.index < '2020-01-01')]/10., 
                               bins = np.arange(-40,47,2.5), density=True)
# Put the information in a dataframe
temps = pd.DataFrame({'hist': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

arr_hist, edges = np.histogram(df_tmin['DATA_VALUE'][(df_tmin.index >= '1901-01-01') & (df_tmin.index < '1930-01-01')]/10., 
                               bins = np.arange(-40,47,2.5), density=True)
# Put the information in a dataframe
temps2 = pd.DataFrame({'hist': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

In [14]:
# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram of Low Temperatures at CHAMPAIGN 3S',
          x_axis_label = 'Temperature (°C)', 
           y_axis_label = 'Fraction of Days')

# Add a quad glyph
p.quad(bottom=0, top=temps['hist'], 
       left=temps['left'], right=temps['right'], 
       fill_color='red', line_color='black', alpha=0.5,legend_label="1991-2020")

p.quad(bottom=0, top=temps2['hist'], 
       left=temps2['left'], right=temps2['right'], 
       fill_color='blue', line_color='black', alpha=0.5, legend_label="1901-1930")

# Show the plot
show(p)

AttributeError: unexpected attribute 'plot_height' to figure, similar attributes are outer_height, height or min_height